<a href="https://colab.research.google.com/github/nrysam/CNN-Car-Parts/blob/main/CNN_based_Car_Part_Identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import necessary libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Get the dataset
!gdown --id 1oO2A6mmoOYRQsrVXvJUdCCvdiJMvPbK2

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1oO2A6mmoOYRQsrVXvJUdCCvdiJMvPbK2
From (redirected): https://drive.google.com/uc?id=1oO2A6mmoOYRQsrVXvJUdCCvdiJMvPbK2&confirm=t&uuid=c32e25ba-f9c2-4e74-aa65-f4ae849ebdbf
To: /content/car parts 50.zip
100% 150M/150M [00:02<00:00, 64.4MB/s]


In [3]:
# Unzip the dataset
!unzip 'car parts 50.zip' -d dataset

Streaming output truncated to the last 5000 lines.
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/040.jpg  
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/041.jpg  
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/042.jpg  
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/043.jpg  
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/044.jpg  
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/045.jpg  
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/046.jpg  
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/047.jpg  
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/048.jpg  
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/049.jpg  
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/050.jpg  
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/051.jpg  
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/052.jpg  
  inflating: dataset/car parts 50/train/LOWER CONTROL ARM/053.jpg

In [4]:
#define the training, validation, testing base directories
train_dir = 'dataset/car parts 50/train'
validation_dir = 'dataset/car parts 50/valid'
test_dir = 'dataset/car parts 50/test'

In [11]:
#Define Target Size, Batch Size, and Class for Generators
TARGET_SIZE = (150,150)
BATCH_SIZE = 32
CLASS_MODE = 'sparse'

In [15]:
def train_val_test_generators(TRAINING_DIR, VALIDATION_DIR, TESTING_DIR):
  """
  Creates the training and validation data generators

  Args:
    TRAINING_DIR (string): directory path containing the training images
    VALIDATION_DIR (string): directory path containing the validation images
    TESTING_DIR (string): directory path containing the testing images
  Returns:
    train_generator, validation_generator, testing_generator: tuple containing the generators
  """
  # Instantiate the ImageDataGenerator class for the training dataset with augmentation
  train_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range = 40,
      width_shift_range=.2,
      height_shift_range=.2,
      shear_range=.2,
      zoom_range=.2,
      horizontal_flip=True,
      fill_mode='nearest'
  )

  # Pass in the appropriate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(
      TRAINING_DIR,
      target_size= TARGET_SIZE,
      batch_size = BATCH_SIZE,
      class_mode = CLASS_MODE
  )

  # Instantiate the ImageDataGenerator class for the validation dataset
  validation_datagen = ImageDataGenerator(rescale = 1./255)

  # Pass in the appropriate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(
      VALIDATION_DIR,
      target_size= TARGET_SIZE,
      batch_size = BATCH_SIZE,
      class_mode = CLASS_MODE
  )

  # Instantiate the ImageDataGenerator class for the testing dataset
  test_datagen = ImageDataGenerator(rescale=1./255)

  # Pass in the appropriate arguments to the flow_from_directory method
  test_generator = test_datagen.flow_from_directory(
      TESTING_DIR,
      target_size= TARGET_SIZE,
      batch_size = BATCH_SIZE,
      class_mode = CLASS_MODE,
      shuffle = False
  )

  return train_generator, validation_generator, test_generator

In [16]:
train_generator, validation_generator, test_generator = train_val_test_generators(train_dir, validation_dir, test_dir)

Found 8739 images belonging to 50 classes.
Found 250 images belonging to 50 classes.
Found 250 images belonging to 50 classes.


In [9]:
#Define the model

def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(50, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['acc'])

    return model

In [10]:
# Save the model
model = create_model()

# Train the model
history = model.fit(train_generator,
                    epochs=100,
                    validation_data=validation_generator)

Epoch 1/100
 22/274 [=>............................] - ETA: 8:30 - loss: 4.0178 - acc: 0.0185

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-86f102162240>", line 5, in <cell line: 5>
    history = model.fit(train_generator,
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1807, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphic_function/polymorphic_function.py", line 832, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/eager/polymorphi

TypeError: object of type 'NoneType' has no len()

In [ ]:
# Plot the chart for accuracy and loss on both training and validation
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

# Accuracy chart
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

# Loss chart
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# Evaluate the Model
results = model.evaluate(test_generator, steps=test_generator.n // test_generator.batch_size)
print('test loss, test acc:', results)

In [ ]:
# Create a prediction from model
predictions = model.predict(test_generator, steps=test_generator.n // test_generator.batch_size)
predicted_classes = np.argmax(predictions, axis=1)

# Compare the predictions against the true labels
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

In [ ]:
# Use scikit-learn to generate a detailed classification report
from sklearn.metrics import classification_report

print(classification_report(true_classes, predicted_classes, target_names=class_labels))

In [ ]:
# Add Confusion matrix to visualize the performance across the different classes
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(15,10))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()